In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
import sqlite3
# descargar el database de kaggle: https://www.kaggle.com/datasets/hugomathien/soccer
address = '../data/database.sqlite'
con = sqlite3.connect(address)

In [3]:
# Creamos un df con todos los datos que nos interesan

df_partidos = pd.read_sql("""SELECT season,
                                date,
                                Country.name AS country_name,                                 
                                HT.team_long_name AS home_team,
                                AT.team_long_name AS away_team,
                                home_team_goal,
                                away_team_goal                                    
                                    FROM Match
                                    JOIN Country on Country.id = Match.country_id
                                    JOIN League on League.id = Match.league_id
                                    LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                                    LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                                WHERE Country.name = "Spain"
                                """, con)

-----
Ver cómo ha quedado el df

In [4]:
pd.options.display.max_columns = None 

In [5]:
df_partidos.head()

,season,date,country_name,home_team,away_team,home_team_goal,away_team_goal
0,2008/2009,2008-08-30 00:00:00,Spain,Valencia CF,RCD Mallorca,3,0
1,2008/2009,2008-08-31 00:00:00,Spain,CA Osasuna,Villarreal CF,1,1
2,2008/2009,2008-08-31 00:00:00,Spain,RC Deportivo de La Coruña,Real Madrid CF,2,1
3,2008/2009,2008-08-31 00:00:00,Spain,CD Numancia,FC Barcelona,1,0
4,2008/2009,2008-08-31 00:00:00,Spain,Racing Santander,Sevilla FC,1,1


In [6]:
# Vemos que hay 1 partido con NaN en las apuestas, por lo que lo quitamos para que no dé problemas
df_partidos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3040 entries, 0 to 3039
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   season          3040 non-null   object
 1   date            3040 non-null   object
 2   country_name    3040 non-null   object
 3   home_team       3040 non-null   object
 4   away_team       3040 non-null   object
 5   home_team_goal  3040 non-null   int64 
 6   away_team_goal  3040 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 166.4+ KB


In [7]:
# Ver cúantas temporadas tenemos
df_partidos['season'].unique()

array(['2008/2009', '2009/2010', '2010/2011', '2011/2012', '2012/2013',
       '2013/2014', '2014/2015', '2015/2016'], dtype=object)

In [8]:
# Lista de equipos
equipos=df_partidos['home_team'].unique()
equipos.sort()
equipos


array(['Athletic Club de Bilbao', 'Atlético Madrid', 'CA Osasuna',
       'CD Numancia', 'CD Tenerife', 'Córdoba CF', 'Elche CF',
       'FC Barcelona', 'Getafe CF', 'Granada CF',
       'Hércules Club de Fútbol', 'Levante UD', 'Málaga CF',
       'RC Celta de Vigo', 'RC Deportivo de La Coruña', 'RC Recreativo',
       'RCD Espanyol', 'RCD Mallorca', 'Racing Santander',
       'Rayo Vallecano', 'Real Betis Balompié', 'Real Madrid CF',
       'Real Sociedad', 'Real Sporting de Gijón', 'Real Valladolid',
       'Real Zaragoza', 'SD Eibar', 'Sevilla FC', 'UD Almería',
       'UD Las Palmas', 'Valencia CF', 'Villarreal CF',
       'Xerez Club Deportivo'], dtype=object)

In [9]:
print("Número de equipos: {}".format(df_partidos['home_team'].nunique()))

Número de equipos: 33


In [10]:
# 1. Vemos que hay un partido con NaN en las apuestas, por lo que lo quitamos para que no dé problemas

# 2. Vemos que en la fecha el horario está vacío en todos los partidos 00:00:00, así que procedemos a quitarlo.

-----

In [11]:
# Quitar el valor NaN
df_partidos=df_partidos.dropna()
df_partidos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3040 entries, 0 to 3039
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   season          3040 non-null   object
 1   date            3040 non-null   object
 2   country_name    3040 non-null   object
 3   home_team       3040 non-null   object
 4   away_team       3040 non-null   object
 5   home_team_goal  3040 non-null   int64 
 6   away_team_goal  3040 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 190.0+ KB


In [12]:
# Quitar hh:mm:ss de 'date'
df_partidos['date']=df_partidos['date'].replace({" 00:00:00":""}, regex=True)
df_partidos.head()

,season,date,country_name,home_team,away_team,home_team_goal,away_team_goal
0,2008/2009,2008-08-30,Spain,Valencia CF,RCD Mallorca,3,0
1,2008/2009,2008-08-31,Spain,CA Osasuna,Villarreal CF,1,1
2,2008/2009,2008-08-31,Spain,RC Deportivo de La Coruña,Real Madrid CF,2,1
3,2008/2009,2008-08-31,Spain,CD Numancia,FC Barcelona,1,0
4,2008/2009,2008-08-31,Spain,Racing Santander,Sevilla FC,1,1


-------


In [13]:
# Creación de la columna 'results'

In [14]:
# Victoria del equipo local = 1
# Victoria del equipo visitante = 2
# Empate = 0

def result (row):
    if row['home_team_goal'] > row['away_team_goal']:
        return 1
    if row['home_team_goal'] == row['away_team_goal']:
        return 0
    if row['home_team_goal'] < row['away_team_goal']:
        return 2

In [15]:
df_partidos['result'] = df_partidos.apply (lambda row: result(row), axis=1)
df_partidos.head(6)

,season,date,country_name,home_team,away_team,home_team_goal,away_team_goal,result
0,2008/2009,2008-08-30,Spain,Valencia CF,RCD Mallorca,3,0,1
1,2008/2009,2008-08-31,Spain,CA Osasuna,Villarreal CF,1,1,0
2,2008/2009,2008-08-31,Spain,RC Deportivo de La Coruña,Real Madrid CF,2,1,1
3,2008/2009,2008-08-31,Spain,CD Numancia,FC Barcelona,1,0,1
4,2008/2009,2008-08-31,Spain,Racing Santander,Sevilla FC,1,1,0
5,2008/2009,2008-08-31,Spain,Real Sporting de Gijón,Getafe CF,1,2,2


----


------

-----------

------

In [16]:
# GUARDAR DF
df_partidos.to_csv('../data/df_partidos.csv')

In [17]:
#Creamos la tabla principal
sql = sqlite3.connect('../data/DB_futbol.db')
cursor = sql.cursor()

lista_valores = df_partidos.values.tolist()

cursor.execute('''CREATE TABLE table_partidos(season TEXT, date TEXT, country_name TEXT, home_team TEXT, away_team TEXT, 
home_team_goal INT, away_team_goal INT, result INT )''')

cursor.executemany("INSERT INTO table_partidos VALUES (?,?,?,?,?,?,?,?)", lista_valores)

sql.commit()
sql.close()